# ETL : Extract, Transform, Load

__Extract:__ read the data, often from mupltiple sources/formats.

__Transform:__ clean and structure the data to suit business needs.

__Load:__ load the data into a database for storage that can be used for future analysis or business use.


In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
import pymysql
pymysql.install_as_MySQLdb()

# Step 1: Extract data into DataFrames

1) __[Enerdata World Oil Consumption Statistical Yearbook](https://yearbook.enerdata.net/oil-products/world-oil-domestic-consumption-statistics.html
)__

2) __[EIA International Coal Consumption](https://www.eia.gov/beta/international/data/browser/#/?pa=0000000000000000000000000000000000000000000000000000000000g&c=ruvvvvvfvtvnvv1vrvvvvfvvvvvvfvvvou20evvvvvvvvvvvvuvs&ct=0&vs=INTL.4411-2-AFG-QBTU.A&vo=0&v=H&start=1990&end=2016)__

In [22]:
# Enerdata data
Enerdata_oil ="Resources/Enerdata World Oil Consumption.csv"
Enerdata_oil_data = pd.read_csv(Enerdata_oil)
Enerdata_oil_data.head()

,Oil products domestic consumption (Mt),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Column1,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2016 - 2017 (%),2000 - 2017 (%/year)
2,World,3090.0,3116.0,3132.0,3132.0,3164.0,3236.0,3317.0,3395.0,3426.0,...,3908.0,3938.0,3970.0,4007.0,4050.0,4121.0,4188.0,4258.0,1.7,1.1
3,OECD,1789.0,1800.0,1840.0,1855.0,1905.0,1919.0,1966.0,1992.0,2001.0,...,1893.0,1861.0,1834.0,1817.0,1800.0,1822.0,1844.0,1864.0,1.1,-0.5
4,G7,1387.0,1381.0,1403.0,1409.0,1435.0,1439.0,1472.0,1485.0,1498.0,...,1365.0,1344.0,1320.0,1309.0,1300.0,1308.0,1312.0,1323.0,0.8,-0.8


In [4]:
# EIA data
EIA_coal ="Resources/EIA_International_Data.csv"
EIA_coal_data = pd.read_csv(EIA_coal)
EIA_coal_data.head()

,Unnamed: 0,Unnamed: 1,1990,1991,1992,1993,1994,1995,1996,1997,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Afghanistan,Quad Btu,0.002187572,0.001958398,0.000166672,0.000145838,0.000125004,0.00010417,6.25E-05,4.17E-05,...,0.005062667,0.007229405,0.010417009,0.015104664,0.03083431,0.025834152,0.028063389,0.031613501,0.028438401,0.03537612
1,Albania,Quad Btu,0.057443992,0.032323962,0.008462611,0.005516766,0.00452589,0.002142433,0.003026187,0.001044436,...,0.001446142,0.001874629,0.004311647,0.004740134,0.00581135,0.006775445,0.002812672,0.003558197,0.003023455,0.002352831
2,Algeria,Quad Btu,0.028251904,0.031484709,0.029095244,0.022629634,0.022966971,0.013493447,0.008939408,0.01523635,...,0.015373496,0.01567704,0.003821761,7.96E-05,5.65E-05,0.00033495,0.001109634,0.001079212,0.000698246,0.000697224
3,American Samoa,Quad Btu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angola,Quad Btu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Step 2: Transform DataFrames

__Enerdata: World Oil Consumption:__

In [11]:
# Enerdata Data Transformation:

# 1) Rename/Organize Columns (years / floattypes)
oil_rename = Enerdata_oil_data.rename(columns={
    'Oil products domestic consumption (Mt)': 'Country',
    'Unnamed: 1' : '(1990)',
    'Unnamed: 2' : '(1991)',
    'Unnamed: 3' : '(1992)',
    'Unnamed: 4' : '(1993)',
    'Unnamed: 5' : '(1994)',
    'Unnamed: 6' : '(1995)',
    'Unnamed: 7' : '(1996)',
    'Unnamed: 8' : '(1997)',
    'Unnamed: 9' : '(1998)',
    'Unnamed: 10' : '(1999)',
    'Unnamed: 11' : '(2000)',
    'Unnamed: 12' : '(2001)',
    'Unnamed: 13' : '(2002)',
    'Unnamed: 14' : '(2003)',
    'Unnamed: 15' : '(2004)',
    'Unnamed: 16' : '(2005)',
    'Unnamed: 17' : '(2006)',
    'Unnamed: 18' : '(2007)',
    'Unnamed: 19' : '(2008)',
    'Unnamed: 20' : '(2009)',
    'Unnamed: 21' : '(2010)',
    'Unnamed: 22' : '(2011)',
    'Unnamed: 23' : '(2012)',
    'Unnamed: 24' : '(2013)',
    'Unnamed: 25' : '(2014)',
    'Unnamed: 26' : '(2015)',
    'Unnamed: 27' : '(2016)',
    'Unnamed: 28' : '(2017)'
})

# 2) Drop unnecessary COLUMNS
oil_drop = oil_rename.drop(columns={'Unnamed: 29', 'Unnamed: 30'})

# 3) Drop Unneccessary/Repetitive ROWS
    #oil_drop2 = oil_drop.drop([0,1,2,3,4,5,6, 60,61,62], axis=0)
oil_drop2 = oil_drop.drop([0,1], axis=0)

# oil_drop.reset_index=()

# 4) Rename/Print new DataFrame
Enerdata_oil_consumption_df = oil_drop2
Enerdata_oil_consumption_df.head()

,Country,(1990),(1991),(1992),(1993),(1994),(1995),(1996),(1997),(1998),...,(2008),(2009),(2010),(2011),(2012),(2013),(2014),(2015),(2016),(2017)
2,World,3090.0,3116.0,3132.0,3132.0,3164.0,3236.0,3317.0,3395.0,3426.0,...,3898.0,3804.0,3908.0,3938.0,3970.0,4007.0,4050.0,4121.0,4188.0,4258.0
3,OECD,1789.0,1800.0,1840.0,1855.0,1905.0,1919.0,1966.0,1992.0,2001.0,...,1959.0,1866.0,1893.0,1861.0,1834.0,1817.0,1800.0,1822.0,1844.0,1864.0
4,G7,1387.0,1381.0,1403.0,1409.0,1435.0,1439.0,1472.0,1485.0,1498.0,...,1420.0,1343.0,1365.0,1344.0,1320.0,1309.0,1300.0,1308.0,1312.0,1323.0
5,BRICS,486.0,506.0,494.0,489.0,460.0,487.0,495.0,517.0,526.0,...,765.0,766.0,802.0,847.0,884.0,912.0,950.0,992.0,1018.0,1058.0
6,Europe,649.0,655.0,653.0,648.0,651.0,662.0,680.0,680.0,689.0,...,650.0,616.0,612.0,586.0,569.0,556.0,549.0,564.0,574.0,587.0


In [16]:
# Enerdata_oil_consumption_df.dtypes

__EIA: International Coal Consumption:__

In [13]:
# EIA Data Transformation

# 1) Rename/Organize Columns (years / floattypes)
coal_rename = EIA_coal_data.rename(columns={
    'Unnamed: 0' : ' Country',
    '1990' : '(1990)', 
    '1991' : '(1991)',
    '1992' : '(1992)',
    '1993' : '(1993)',
    '1994' : '(1994)',
    '1995' : '(1995)', 
    '1996' : '(1996)',
    '1997' : '(1997)',
    '1998' : '(1998)',
    '1999' : '(1999)', 
    '2000' : '(2000)',
    '2001' : '(2001)',
    '2002' : '(2002)',
    '2003' : '(2003)',
    '2004' : '(2004)',
    '2005' : '(2005)',
    '2006' : '(2006)',
    '2007' : '(2007)',
    '2008' : '(2008)',
    '2009' : '(2009)',
    '2010' : '(2010)',
    '2011' : '(2011)',
    '2012' : '(2012)',
    '2013' : '(2013)',
    '2014' : '(2014)',
    '2015' : '(2015)',
    '2016' : '(2016)'
    
})
coal_rename


# 2) Drop Unnecessary COLUMNS (Quad Btu)
coal_drop = coal_rename.drop(columns={"Unnamed: 1"})
#coal_drop.reset_index=()
#coal_drop

# 3) Rename/Print new DataFrame
EIA_coal_consumption_df = coal_drop
EIA_coal_consumption_df.head()

,Country,(1990),(1991),(1992),(1993),(1994),(1995),(1996),(1997),(1998),...,(2007),(2008),(2009),(2010),(2011),(2012),(2013),(2014),(2015),(2016)
0,Afghanistan,0.002187572,0.001958398,0.000166672,0.000145838,0.000125004,0.00010417,6.25E-05,4.17E-05,4.17E-05,...,0.005062667,0.007229405,0.010417009,0.015104664,0.03083431,0.025834152,0.028063389,0.031613501,0.028438401,0.03537612
1,Albania,0.057443992,0.032323962,0.008462611,0.005516766,0.00452589,0.002142433,0.003026187,0.001044436,0.00131224,...,0.001446142,0.001874629,0.004311647,0.004740134,0.00581135,0.006775445,0.002812672,0.003558197,0.003023455,0.002352831
2,Algeria,0.028251904,0.031484709,0.029095244,0.022629634,0.022966971,0.013493447,0.008939408,0.01523635,0.017653926,...,0.015373496,0.01567704,0.003821761,7.96E-05,5.65E-05,0.00033495,0.001109634,0.001079212,0.000698246,0.000697224
3,American Samoa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angola,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# EIA_coal_consumption_df.dtypes

> __Connect to local database__

In [19]:
connection_string = "root:root@127.0.0.1/oil_coal_consumption_db"
engine = create_engine(f'mysql://{connection_string}')

In [20]:
# Confirm Tables
engine.table_names()

InternalError: (pymysql.err.InternalError) (1049, "Unknown database 'oil_coal_consumption_db'") (Background on this error at: http://sqlalche.me/e/2j85)

# Step 3: Load DataFrames into database


In [21]:
# Enerdata_oil_consumption_df.to_sql(name='eia_coal', con=engine, if_exists='append', index=False)


In [ ]:
# premise_transformed.to_sql(name='enerdata_oil', con=engine, if_exists='append', index=True)

In [ ]:
##### E: Data may come from disparate sources, such as: 
# > CSV
# > JSON
# > HTML
# > MySQL
# > SQL
# > Spreadsheets
# > etc.


##### T :
# > Data Cleaning
# > Summarization
# > Selection
# > Joining
# > Filtering
# > Aggregating
# > etc.



##### L:
# > Can be a relational or non-relational database
# > Can be local or in the cloud
# > Can be a data lake or data warehouse